In [2]:
# problem 파일 경로 Desktop/project/code_similarity/open/code

# problem 폴더 안 예제 코드 확장자명 변경
import os
from tqdm import tqdm
import sys
import tensorflow as tf
import json
import datetime

# .txt 병합
import pandas as pd
import numpy as np
import sklearn

#keras model import
import keras

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from keras import Input, layers
from tensorflow.keras.layers import Embedding, LSTM, Input, Lambda,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
from tensorflow.keras.optimizers import Adam
#model save & load
from keras.models import load_model

In [4]:
train_data = pd.read_csv("./open_code_train_220524.csv", index_col = 0)

In [5]:
print(train_data.head())
print(train_data['similar'].value_counts())
print(train_data.isnull().values.any())

                                               code1  c1_problem_name  \
0  n, m, k = map(int, input().split(" "))\na = [i...              117   
1                 N = int(input())\nprint(int(N**2))              278   
2  x, n = map(int, input().split())\nlis =  []\ni...              128   
3  import sys\ninput = sys.stdin.readline\ndef ma...               71   
4  import sys\nINF = 1 << 60\nMOD = 10**9 + 7 \ns...               75   

                                               code2  c2_problem_name  similar  
0            a=int(input())\nprint(a*6.283185307178)              167        0  
1  from heapq import heappush, heappop\nimport sy...              241        0  
2  from collections import defaultdict,Counter\nh...               89        0  
3  word=input()\nlast_letter=word[int(len(word))-...               71        1  
4  import sys\nimport math\nimport collections\ni...              118        0  
1    30169
0    29831
Name: similar, dtype: int64
False


In [6]:
test_data = pd.read_csv("./test.csv")

In [7]:
test_data = test_data.head(10000)

In [8]:
train_data.head()

,code1,c1_problem_name,code2,c2_problem_name,similar
0,"n, m, k = map(int, input().split("" ""))\na = [i...",117,a=int(input())\nprint(a*6.283185307178),167,0
1,N = int(input())\nprint(int(N**2)),278,"from heapq import heappush, heappop\nimport sy...",241,0
2,"x, n = map(int, input().split())\nlis = []\ni...",128,"from collections import defaultdict,Counter\nh...",89,0
3,import sys\ninput = sys.stdin.readline\ndef ma...,71,word=input()\nlast_letter=word[int(len(word))-...,71,1
4,import sys\nINF = 1 << 60\nMOD = 10**9 + 7 \ns...,75,import sys\nimport math\nimport collections\ni...,118,0


In [9]:
train_data.drop(['c1_problem_name','c2_problem_name'], axis = 1, inplace= True)

In [10]:
print(type(train_data['code1']))
print(type(train_data['code2']))
print(train_data.head())

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
                                               code1  \
0  n, m, k = map(int, input().split(" "))\na = [i...   
1                 N = int(input())\nprint(int(N**2))   
2  x, n = map(int, input().split())\nlis =  []\ni...   
3  import sys\ninput = sys.stdin.readline\ndef ma...   
4  import sys\nINF = 1 << 60\nMOD = 10**9 + 7 \ns...   

                                               code2  similar  
0            a=int(input())\nprint(a*6.283185307178)        0  
1  from heapq import heappush, heappop\nimport sy...        0  
2  from collections import defaultdict,Counter\nh...        0  
3  word=input()\nlast_letter=word[int(len(word))-...        1  
4  import sys\nimport math\nimport collections\ni...        0  


In [11]:
tokenizer = Tokenizer() # Tokenizer 객체 생성

In [12]:
tokenizer.fit_on_texts(train_data['code1']) # 단어 인덱스 구축
tokenizer.fit_on_texts(train_data['code2']) # 단어 인덱스 구축

In [13]:
sequences_1 = tokenizer.texts_to_sequences(train_data['code1']) # 문자열을 정수 인덱스의 리스트로 변환한다. 
sequences_2 = tokenizer.texts_to_sequences(train_data['code2'])

In [14]:
train_val = train_data['similar']
train_val.head()

0    0
1    0
2    0
3    1
4    0
Name: similar, dtype: int64

In [15]:
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print(vocab_size)

12440


In [18]:
#가장 긴 sequence의 길이
MAX_SEQUENCE_LENGTH = max([len(seq) for seq in sequences_1 + sequences_2])
ABR__SEQUENCE_LENGTH = round(sum(map(len, sequences_1))/len(sequences_1))
print('코드의 최대 길이 : {}'.format(max([len(seq) for seq in sequences_1 + sequences_2])))
print('코드의 평균길이 : {}'.format(sum(map(len, sequences_1))/len(sequences_1)))
print('코드의 평균길이 : %d'% ABR__SEQUENCE_LENGTH)

코드의 최대 길이 : 803
코드의 평균길이 : 63.076366666666665
코드의 평균길이 : 63


In [31]:
MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH
ABR__SEQUENCE_LENGTH = 256

In [32]:
X_train_1 = pad_sequences(sequences_1, maxlen=ABR__SEQUENCE_LENGTH)
X_train_2 = pad_sequences(sequences_2, maxlen=ABR__SEQUENCE_LENGTH)
print(X_train_1.shape)
print(X_train_2.shape)
print(type(X_train_2))

(60000, 256)
(60000, 256)
<class 'numpy.ndarray'>


In [33]:
# Model variables
batch_size = 512
epoch = 10
vocab_size = vocab_size
MAX_SEQUENCE_LENGTH = MAX_SEQUENCE_LENGTH
ABR__SEQUENCE_LENGTH = ABR__SEQUENCE_LENGTH
vocab_size = vocab_size
emb_output_dim = 128
DROPOUT_RATIO = 0.25

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
input_tensor1 = layers.Input(shape= (256,), dtype = 'int32', name = 'code1')
input_tensor2 = layers.Input(shape= (256,), dtype = 'int32', name = 'code2')

# Embedding layers
embedding_layer = layers.Embedding(input_dim = vocab_size,
                                   output_dim = emb_output_dim,
                                   input_length = ABR__SEQUENCE_LENGTH,
                                   mask_zero = True,
                                   trainable = False)

# Embedded of the inputs
encoded_1 = embedding_layer(input_tensor1)
encoded_2 = embedding_layer(input_tensor2)

# define lstm for sentence encode
encoded_LSTM1 = layers.LSTM(64, activation='tanh')(encoded_1)
encoded_LSTM2 = layers.LSTM(64, activation='tanh')(encoded_2)

#dropout layer
drop1 = layers.Dropout(DROPOUT_RATIO)(encoded_LSTM1)
drop2 = layers.Dropout(DROPOUT_RATIO)(encoded_LSTM2)

# Calculates the distance as defined by the MaLSTM model
#concats = layers.concatenate([encoded_LSTM1,encoded_LSTM2], axis = -1)

#함수형 API 모델 을 구성할 때 임의 의 Lambda표현식을 사용할 수 있도록 레이어 생성
main_malstm_distance = layers.Lambda(lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([drop1,drop2])

#output
#answer = layers.Dense(1, activation='sigmoid')(main_malstm_distance)
# 최종 아웃풋을 어떤것으로 지정할것인가?

#model
malstm = Model([input_tensor1, input_tensor2], outputs = main_malstm_distance)

#model compile 
malstm.compile(loss='mean_squared_logarithmic_error', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [34]:
malstm.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 code1 (InputLayer)             [(None, 256)]        0           []                               
                                                                                                  
 code2 (InputLayer)             [(None, 256)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 256, 128)     1592320     ['code1[0][0]',                  
                                                                  'code2[0][0]']                  
                                                                                                  
 lstm_4 (LSTM)                  (None, 64)           49408       ['embedding_2[0][0]']      

In [35]:
# early_stop
early_stopping = EarlyStopping(monitor="val_loss",patience = 10, mode="min")

In [ ]:
history = malstm.fit([X_train_1,X_train_2],
                   train_val,
                   verbose=1,
                   callbacks=[early_stopping],
                   validation_split=0.2,
                   epochs=150,
                   batch_size = batch_size)

Epoch 1/150
 4/94 [>.............................] - ETA: 17:16 - loss: 0.1324 - accuracy: 0.5078

In [ ]:
model.save('./model_smilarity_MSLE_803')

In [ ]:
# Plot accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# # Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
#test data 정수
tokenizer.fit_on_texts(test_data['code1']) # 단어 인덱스 구축
tokenizer.fit_on_texts(test_data['code2']) # 단어 인덱스 구축

test_sequences_1 = tokenizer.texts_to_sequences(test_data['code1']) # 문자열을 정수 인덱스의 리스트로 변환한다. 
test_sequences_2 = tokenizer.texts_to_sequences(test_data['code2'])

X_test_1 = pad_sequences(test_sequences_1, maxlen=ABR__SEQUENCE_LENGTH)
X_test_2 = pad_sequences(test_sequences_2, maxlen=ABR__SEQUENCE_LENGTH)

print(X_test_1.shape)
print(X_test_2.shape)
print(type(X_test_1))

In [ ]:
# 모델 추론
preds = malstm.predict(test_data['code1'], test_data['code2'])

In [ ]:
threshold = 0.5
preds = np.where(preds>self.threshold, 1, 0)

In [ ]:
# submission 저장
submission = pd.read_csv('./sample_submission.csv')
submission['similar'] = preds
submission.to_csv('./submission.csv', index=False)